In [3]:
import os
print(os.getcwd())

C:\Users\em18921\Documents\Sentinels_indicator\Codes_folder


In [5]:
import glob
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore
from statsmodels.tsa.seasonal import STL
import re

# Sorting files by batch numbers so that they are merged as per batch number file name

In [8]:
import re

# Define a function to get the batch number from the filename
def get_batch_number(file_paths):
    # Step 1: Define the pattern to match the batch number
    # Explanation of pattern:
    # - `batch_`: Matches the text 'batch_' exactly
    # - `(\d+)`: Matches one or more digits and captures them as a group
    pattern = r'batch_(\d+)'
    
    # Step 2: Use regex search to find the batch number in the filename
    # `re.search()` returns a match object if it finds the pattern, or None if it doesn’t
    match = re.search(pattern, file_paths)
    
    # Step 3: Check if a match was found
    if match:
        # If a match is found, extract the captured group (the batch number)
        # Convert it to an integer because we want to sort by batch number as a number, not as text
        batch_number = int(match.group(1))  # match.group(1) is the captured number as a string
        return batch_number
    else:
        # If no match is found (no batch number in the filename), return a high value
        # Using `float('inf')` here means these files will appear last in a sorted list
        return float('inf')

# Define file paths and sort file paths

In [11]:
import glob

# Define input file path 
input_directory = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data'

# Get all the matching filenames from the directory. Matching file name
file_paths = glob.glob(os.path.join(input_directory, 'monthly data_geopoltiks_monthly_batch_*.csv'))

#for file_path in file_paths:
    #print(file_path)

sorted_file_paths = sorted(file_paths, key=get_batch_number)

for sorted_file_path in sorted_file_paths:
    print(sorted_file_path)

C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_1.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_2.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_3.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_4.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_5.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_6.csv
C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_7.csv
C:\Users\em18921\Doc

# Process each file path in sorted_file_paths

In [13]:
import datetime
import datetime as dt
import pandas as pd

# Define counter
counter = 1

# Start the loop for sorted_file_paths
for sorted_file_path in sorted_file_paths:
    print(f"processing file: {sorted_file_path}")

    # Read the file from file path
    df = pd.read_csv(sorted_file_path)

    # Drop isPArtial column from the dataframe
    df = df.drop('isPartial', axis=1, errors='ignore')

    # Remove any leading and trailing whitespaces from the column
    df.columns = df.columns.str.strip()

    # If date column is present in the dataframe convert it into day, month and year colum
    df['date'] = pd.to_datetime(df['date'])

    df['Year'] = df['date'].dt.year
    df['Month'] = df['date'].dt.month
    df['Day'] = df['date'].dt.day

    # Column for batch number
    df['batch_number'] = f'apidata_{counter}'

    # Output the dataset
    output_dir = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files'
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, f'geopolitiks_mon_us_batch_{counter}.csv')
    df.to_csv(output_file_path, index=False)

    print(f'file_{counter} saved in {output_file_path}')
    
    counter += 1

processing file: C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_1.csv
file_1 saved in C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_1.csv
processing file: C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_2.csv
file_2 saved in C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_2.csv
processing file: C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly raw data\monthly data_geopoltiks_monthly_batch_3.csv
file_3 saved in C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_3.csv
processing file: C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geop

# Concat all the files in the file path

In [15]:
input_directory = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files'

input_file_paths = glob.glob(os.path.join(input_directory, 'geopolitiks_mon_us_batch_*.csv'))

file_paths = sorted(input_file_paths, key=get_batch_number)

# define an empty list to hold melted dataframe
melted_df = []

for sorted_file_path in file_paths:
    df = pd.read_csv(sorted_file_path)

    # melt the dataframe
    df_long = df.melt(id_vars=['date', 'Year', 'Month', 'Day', 'batch_number'],
                      value_name='SVI',
                      var_name='keyword_name')

    # Append the melted dfs to the list
    melted_df.append(df_long)

    print(f"processed file path {sorted_file_path}")
    
# Convert the melted_df into dataframe
df_combined = pd.concat(melted_df, ignore_index=True)

# output the dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopoltiks_mon_merged_set.csv'
df_combined.to_csv(output_file_path, index=False)

df_combined

processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_1.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_2.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_3.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_4.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_5.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopolitiks_mon_us_batch_6.csv
processed file path C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopoli

,date,Year,Month,Day,batch_number,keyword_name,SVI
0,2004-01-01,2004,1,1,apidata_1,act of war,20
1,2004-02-01,2004,2,1,apidata_1,act of war,27
2,2004-03-01,2004,3,1,apidata_1,act of war,26
3,2004-04-01,2004,4,1,apidata_1,act of war,27
4,2004-05-01,2004,5,1,apidata_1,act of war,27
...,...,...,...,...,...,...,...
62491,2024-08-01,2024,8,1,apidata_83,anti war,0
62492,2024-09-01,2024,9,1,apidata_83,anti war,0
62493,2024-10-01,2024,10,1,apidata_83,anti war,0
62494,2024-11-01,2024,11,1,apidata_83,anti war,0


# Select unique name of keyword_name and year

In [44]:
# Merge datasets

path_1 = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopoltiks_mon_merged_set.csv'

df_1 = pd.read_csv(path_1)

# Keep keyword_name
keep_keyword_name = [ 'act of war','air battles','air to air combat','air warfare','american revolution', 'cold war', 'combat', 'conflict', 'coup','drone warfare', 
        'guerilla war', 'guerrilla warfare', 'hijack war', 'insurgents', 'insurrection', 'invade', 'invasion', 'iran israel conflict', 'iran israel proxy conflict', 
        'iran attack israel', 'iran attack israel israeli', 'iran us tensions', 'israel hamas war', 
        'israel hezbollah conflict', 'israeli lebanese conflict', 'lebanon war', 'massacre', 'military coup',
        'post war', 'red scare', 'russo-ukrainian war', 'submarine warfare', 'the insurgents', 
        'total war', 'trench warfare', 'war', 'war blockade', 'war blockades', 'war clashes', 'war effort', 'war efforts', 'war hostage','war hostages',
        'war rebels', 'war suffering', 'war weapon', 'war weapons', 'warfare', 'warhead', 'warheads', 'world war', 'missile launch',
        'aerial support', 'air force', 'armed forces', 'army', 'bomb', 'bombings', 'bombs', 'central intelligence agency', 'cia', 'cruise missile', 
        'drone strike', 'drone strikes', 'hypersonic missile', 'idf', 'militarism', 'military', 'military advance', 'military alliance', 'military and foreign affairs', 
        'military arms', 'military base', 'military branch', 'military dictatorship', 'military drone', 'military drones', 'military fails', 'military news',
        'military operation', 'military press', 'military strategy', 'military strike', 'military wars strike', 'missile test',
        'mobilize troops', 'nato', 'nato troops', 'naval warfare','navy', 'offensive military', 'pentagon', 
        'russian troops', 'soldiers surrender', 'troop', 'troops', 'warriors', 'atomic bomb', 'hydrogen bomb', 'h-bomb', 'hypersonic nuclear missile',
        'manhattan project', 'nuclear bomb', 'nuclear missile', 'nuclear missiles', 'nuclear power plants', 'nuclear powered missile', 'nuclear test',
        'nuclear threats', 'nuclear war', 'nuclear weapon', 'weapons of mass destruction', 'nukemap', 
        'nuclear warhead', 'nuclear warheads', 'act of terrorism', 'domestic terrorism', 'international terrorism',
        'militia', 'paris attacks', 'russian invasion', 'russian military buildup', 'terrorism', 
        'terrorist', 'terrorist act', 'terrorist attack', 'terrorist attacks', 'terrorist killed', 'cold war threats', 'threats', 'disrupt peace',
        'peace disruption', 'breach of the peace', 'hostage crisis', 'plane hijack', 'assassination', 'embargo', 'economic sanctions', 'embargo act',
        'political risk', 'geopolitical risks', 'oil crisis', 'oil embargo', 'conscription', 'military buildup', 'military build-up', 'military drills',
        'military motivation', 'refugee crisis', 'refugees', 'genocide', 'missile crisis', 'naval blockades', 'military clashes',
        'arms trade treaty', 'sanctions act of war', 'surgical strike', 'trade embargo','trade sanctions', 
        'troops at the border', 'truce agreement', 'us and china tensions', 'us china tensions', 'war neutrality', 'war refugee', 'what are sanctions in war', 'tariff', 
        'tariffs', 'tariff news', 'trade tariff', 'trade war', 'brink of war', 'brink of world war', 'on the brink of civil war', 'on the brink of world war 3',
        'military combat', 'outbreak of war', 'outbreak of world war', 'rising tensions', 'war tensions', 'war imminent', 'war is imminent',
        'start of war', 'the great fear', 'the great war', 'war scare', 'an act of war', 'attack on military base', 'escalation of the war',
        'escalation of war', 'arms race', 'allied powers', 'allies of world war', 'central powers', 'fascism', 'geopolitical', 'geopolitical boundaries',
        'geopolitical climate', 'geopolitical community', 'geopolitical conflict', 'geopolitical events', 'geopolitical news', 'geopolitical situation',
        'geopolitical strategy', 'geopolitical tensions', 'geopolitics', 'gulf of tonkin','human migration', 'nationalism', 'nazism', 'protest',
        'triple alliance', 'war news', 'war powers act', 'world order', 'anti-war movement', 'armistice', 
        'calls for end of war', 'ceasefire', 'christmas truce', 'civil war truce', 'global peace index', 'god', 'humanitarian aid', 
        'neutral countries', 'neutral country', 'patriot act', 'peace', 'peace corps', 
        'peace symbols', 'peace process', 'peace talk', 'peace talks', 'peace tea', 'peace treaty', 'religious text', 
        'small arms treaty', 'treaty', 'arms treaty', 'world peace', 'alliance', 'diplomacy', 'foreign  policy', 'justice', 'policy', 'anti war']


df_1 = df_1[df_1['keyword_name'].isin(keep_keyword_name)]


df_1 = df_1.drop_duplicates(subset=['keyword_name', 'Year', 'Month', 'Day'], keep="first")

df_unique = df_1[['keyword_name', 'Year']].drop_duplicates()

df_unique['Year_number'] = df_unique['Year'] - 2003

# Define the year range, i.e 2003 to 2025
all_years = range(2003, 2025)

# Get all unique keyword_name data
df_unique_key = df_1['keyword_name'].unique()

# Create a cross dataset with unique keyword_name and all_years
cross_data = pd.DataFrame([(keyword_name, Year) for keyword_name in df_unique_key for Year in all_years],
                         columns = ['keyword_name', 'Year'])

# Merge the cross data with df_unique and keep the indicator on
df_merge = pd.merge(df_unique, cross_data, on=['keyword_name', 'Year'], how='right', indicator=True)

# Output dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopoltiks_mon_final_dataset.csv'
df_1.to_csv(output_file_path, index=False)

# Output dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\csvmeta_data_4.csv'
df_merge.to_csv(output_file_path, index=False)

#df_1
#df_merge
print(len(keep_keyword_name))

246


# Merge final dataset with week_number type dataset

In [79]:
path_1 = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopoltiks_mon_final_dataset.csv'
df_1 = pd.read_csv(path_1)
# Drop Day column from the dataset
df_1['date'] = pd.to_datetime(df_1['date'])

path_2 = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\original_monthly_date.csv'
df_2 = pd.read_csv(path_2)

# Convert date to Year and Month and drop date
df_2['date'] = pd.to_datetime(df_2['date'], format="%d/%m/%Y")
df_2['Month'] = df_2['date'].dt.month
df_2['Year'] = df_2['date'].dt.year
df_2['Day'] = df_2['date'].dt.day
df_2 = df_2.drop('date', axis=1)
df_2['month_number'] = df_2.groupby('Year').cumcount() + 1

df_merge = pd.merge(df_1, df_2, on=['Year', 'Month', 'Day'], how='left')
df_merge

output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_monthly\geopolitiks monthly output files\geopoltiks_mon_final_dataset_2.csv'
df_merge.to_csv(output_file_path, index=False)

print(f"Length of original dataset {df_1.shape[0]}")
print(f"Length of dataset after merge is {df_merge.shape[0]}")
df_2

Length of original dataset 61740
Length of dataset after merge is 61740


,time_series_period_week,Month,Year,Day,month_number
0,1,1,2004,1,1
1,2,2,2004,1,2
2,3,3,2004,1,3
3,4,4,2004,1,4
4,5,5,2004,1,5
...,...,...,...,...,...
247,248,8,2024,1,8
248,249,9,2024,1,9
249,250,10,2024,1,10
250,251,11,2024,1,11
